## CIFAR-10
El dataset CIFAR-10 (Canadian Institute For Advanced Research) es un conjunto de datos ampliamente utilizado en el campo del aprendizaje automático y la visión por computadora para tareas de clasificación de imágenes. Fue recopilado por Alex Krizhevsky, Vinod Nair y Geoffrey Hinton.

<ul>
<li>Cantidad de Imágenes: El conjunto de datos contiene 60,000 imágenes a color en total.</li>
<ul>
<li>50,000 imágenes para el conjunto de entrenamiento.</li>
<li>10,000 imágenes para el conjunto de prueba.</li>
<li>Resolución de las Imágenes: Cada imagen es de 32x32 píxeles.</li>
</ul>

<li>Número de Clases: El conjunto de datos está dividido en 10 clases, con 6,000 imágenes por clase.</li>
</ul>


Las 10 clases representan objetos del mundo real y son:
<ul>
<li>Aviones (airplane)</li>
<li>Automóviles (automobile)</li>
<li>Pájaros (bird)</li>
<li>Gatos (cat)</li>
<li>Ciervos (deer)</li>
<li>Perros (dog)</li>
<li>Ranas (frog)</li>
<li>Caballos (horse)</li>
<li>Barcos (ship)</li>
<li>Camiones (truck)</li>
</ul>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA

In [3]:
# Cargar el dataset CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
# Normalizar las imágenes a valores entre 0 y 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [5]:
# Reducir número de elementos
X_train = X_train[:1000]
X_test = X_test[:250]
y_train = y_train[:1000]
y_test = y_test[:250]

In [6]:
# Convertir las etiquetas a formato one-hot
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [7]:
model = models.Sequential([
    # Conv2D Es una capa de convolución que aplica 32 filtros de 3x3 sobre la imagen de entrada.
    # input_shape=(32, 32, 3): Especifica la forma de las imágenes de entrada (32x32 píxeles, 3 canales de color).
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    # MaxPooling2D Reduce la dimensionalidad de la imagen tomando el valor máximo en cada bloque de 2x2 píxeles, reduciendo así las dimensiones espaciales de la imagen a la mitad.
    layers.MaxPooling2D((2, 2)),
    # Similar a la primera capa Conv2D, pero con 64 filtros en lugar de 32
    layers.Conv2D(64, (3, 3), activation='relu'),
    # Igual que la primera capa MaxPooling2D, reduce las dimensiones espaciales a la mitad nuevamente.
    layers.MaxPooling2D((2, 2)),
    # Otra capa de convolución con 64 filtros de 3x3.
    layers.Conv2D(64, (3, 3), activation='relu'),
    # Flatten: Convierte la salida 2D de las capas anteriores en un vector 1D para que pueda ser procesado por capas densamente conectadas.
    layers.Flatten(),
    # Dense para establecer una capa completamente conectada con 64 unidades (neuronas).
    layers.Dense(64, activation='relu'),
    # Dense para establcer una capa completamente conectada con 10 unidades.
    # Usa la función de activación Softmax para producir una distribución de probabilidad sobre las 10 clases de salida
    layers.Dense(10, activation='softmax')
])

# Convolución y MaxPooling: Las capas de convolución extraen características importantes de las imágenes, mientras que las capas de MaxPooling reducen la dimensionalidad espacial y la cantidad de parámetros, ayudando a prevenir el sobreajuste.
# Aplanamiento: La capa Flatten convierte la salida de las capas convolucionales en un vector 1D.
# Capas densas: Las capas Dense realizan la clasificación final basándose en las características extraídas.
# Salida Softmax: La última capa produce una distribución de probabilidad sobre las posibles clases de salida.

In [8]:
# Compilar el modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/10
16/16 [==============================] - 3s 92ms/step - loss: 2.2979 - accuracy: 0.1100 - val_loss: 2.2771 - val_accuracy: 0.1480
Epoch 2/10
16/16 [==============================] - 1s 68ms/step - loss: 2.2415 - accuracy: 0.1590 - val_loss: 2.1845 - val_accuracy: 0.1920
Epoch 3/10
16/16 [==============================] - 1s 68ms/step - loss: 2.1212 - accuracy: 0.2010 - val_loss: 2.0839 - val_accuracy: 0.1960
Epoch 4/10
16/16 [==============================] - 1s 67ms/step - loss: 2.0196 - accuracy: 0.2480 - val_loss: 2.0474 - val_accuracy: 0.2200
Epoch 5/10
16/16 [==============================] - 1s 66ms/step - loss: 1.9321 - accuracy: 0.2980 - val_loss: 1.9639 - val_accuracy: 0.2440
Epoch 6/10
16/16 [==============================] - 1s 78ms/step - loss: 1.8247 - accuracy: 0.3320 - val_loss: 1.9956 - val_accuracy: 0.2680
Epoch 7/10
16/16 [==============================] - 2s 114ms/step - loss: 1.7797 - accuracy: 0.3620 - val_loss: 1.8582 - val_accuracy: 0.3320
Epoch 8/10
1

In [10]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')


8/8 [==============================] - 0s 11ms/step - loss: 1.7724 - accuracy: 0.3640
Test accuracy: 0.36399999260902405


## Escalamiento de los datos con SVM y RandomForest

In [11]:
# Aplanar las imágenes
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Escalar los píxeles de las imágenes al rango de 0 a 1
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)

In [12]:
# Convertir las etiquetas one-hot a etiquetas numéricas simples
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

In [13]:
# Crear un modelo SVC
svm_model = SVC()

In [14]:
# Entrenar el modelo SVC
svm_model.fit(X_train_scaled, y_train)

SVC()

In [15]:
# Predecir con el modelo SVC
svm_pred = svm_model.predict(X_test_scaled)

In [16]:
# Calcular la precisión del modelo SVC
svm_accuracy = accuracy_score(y_test, svm_pred)
print("Accuracy of SVM:", svm_accuracy)

Accuracy of SVM: 0.328


In [17]:
# Crear un modelo Random Forest
rf_model = RandomForestClassifier()

In [18]:
# Entrenar el modelo Random Forest
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [19]:
# Predecir con el modelo Random Forest
rf_pred = rf_model.predict(X_test_scaled)

In [20]:
# Calcular la precisión del modelo Random Forest
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Accuracy of Random Forest:", rf_accuracy)

Accuracy of Random Forest: 0.316


## Aumento de datos con una red neuronal densa

In [21]:
# Normalizar las imágenes a valores entre 0 y 1 (No necesario volver a hacer)
#X_train = X_train.astype('float32') / 255.0
#X_test = X_test.astype('float32') / 255.0

In [22]:
# Definir el generador de datos de imágenes con aumentos
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [23]:
# Ajustar el generador al conjunto de entrenamiento
datagen.fit(X_train)

In [24]:
# Definir la arquitectura de la red neuronal densa
model = models.Sequential([
    layers.Flatten(input_shape=(32, 32, 3)),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Agregar dropout para regularización
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

In [25]:
# Compilar el modelo
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [27]:
# Entrenar el modelo con aumento de datos
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32,  # número de lotes por época
                    epochs=5,  # número de épocas
                    validation_data=(X_test, y_test))

Epoch 1/5
31/31 [==============================] - 1s 46ms/step - loss: 2.2834 - accuracy: 0.1240 - val_loss: 2.2606 - val_accuracy: 0.1720
Epoch 2/5
31/31 [==============================] - 2s 74ms/step - loss: 2.2417 - accuracy: 0.1620 - val_loss: 2.1818 - val_accuracy: 0.2080
Epoch 3/5
31/31 [==============================] - 2s 58ms/step - loss: 2.1810 - accuracy: 0.1580 - val_loss: 2.1136 - val_accuracy: 0.1680
Epoch 4/5
31/31 [==============================] - 1s 35ms/step - loss: 2.1480 - accuracy: 0.1850 - val_loss: 2.1047 - val_accuracy: 0.2120
Epoch 5/5
31/31 [==============================] - 1s 35ms/step - loss: 2.1109 - accuracy: 0.1920 - val_loss: 2.0778 - val_accuracy: 0.2200


In [28]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

8/8 [==============================] - 0s 5ms/step - loss: 2.0778 - accuracy: 0.2200
Test accuracy: 0.2199999988079071


## Reducción de dimensionalidad y una red neuronal convolucional

In [ ]:
# Normalizar las imágenes a valores entre 0 y 1 ( No es necesario volver a hacerlo )
#X_train = X_train.astype('float32') / 255.0
#X_test = X_test.astype('float32') / 255.0

In [29]:
# Aplanar las imágenes para PCA
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

In [30]:
# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=100)  # Reducir a 100 componentes principales
X_train_pca = pca.fit_transform(X_train_flattened)
X_test_pca = pca.transform(X_test_flattened)

In [31]:
# Redimensionar las imágenes para la entrada de la red neuronal convolucional
X_train_cnn = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test_cnn = X_test.reshape(X_test.shape[0], 32, 32, 3)

In [32]:
# Crear y compilar el modelo para datos originales
model_cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [33]:
# Compilar el modelo
model_cnn.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [34]:
X_train_cnn.shape

(1000, 32, 32, 3)

In [35]:
y_train.shape

(1000,)

In [37]:
# Entrenar el modelo con datos originales
history_cnn = model_cnn.fit(X_train_cnn, y_train, epochs=5, batch_size=64, validation_data=(X_test_cnn, y_test))


Epoch 1/5
16/16 [==============================] - 2s 127ms/step - loss: 2.3013 - accuracy: 0.1120 - val_loss: 2.3044 - val_accuracy: 0.0800
Epoch 2/5
16/16 [==============================] - 1s 61ms/step - loss: 2.3012 - accuracy: 0.1120 - val_loss: 2.3046 - val_accuracy: 0.0800
Epoch 3/5
16/16 [==============================] - 1s 60ms/step - loss: 2.3000 - accuracy: 0.1120 - val_loss: 2.3024 - val_accuracy: 0.0800
Epoch 4/5
16/16 [==============================] - 1s 59ms/step - loss: 2.2981 - accuracy: 0.1120 - val_loss: 2.2999 - val_accuracy: 0.0800
Epoch 5/5
16/16 [==============================] - 1s 59ms/step - loss: 2.2941 - accuracy: 0.1100 - val_loss: 2.2938 - val_accuracy: 0.0800


In [38]:
X_test_cnn.shape

(250, 32, 32, 3)

In [39]:
y_test.shape

(250,)

In [40]:
# Evaluar el modelo con datos originales
test_loss_cnn, test_accuracy_cnn = model_cnn.evaluate(X_test_cnn, y_test)
print("Accuracy of model without PCA:", test_accuracy_cnn)

8/8 [==============================] - 0s 10ms/step - loss: 2.2938 - accuracy: 0.0800
Accuracy of model without PCA: 0.07999999821186066


In [41]:
# Crear y compilar el modelo para datos reducidos por PCA
model_pca = models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(100,)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [42]:
model_pca.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [43]:
X_train_pca.shape

(1000, 100)

In [44]:
y_train.shape

(1000,)

In [46]:
# Entrenar el modelo con datos reducidos por PCA
history_pca = model_pca.fit(X_train_pca, y_train, epochs=5, batch_size=64, validation_data=(X_test_pca, y_test))


Epoch 1/5
16/16 [==============================] - 0s 20ms/step - loss: 2.2236 - accuracy: 0.1850 - val_loss: 2.1810 - val_accuracy: 0.2560
Epoch 2/5
16/16 [==============================] - 0s 11ms/step - loss: 2.1345 - accuracy: 0.2230 - val_loss: 2.0873 - val_accuracy: 0.2680
Epoch 3/5
16/16 [==============================] - 0s 12ms/step - loss: 2.0481 - accuracy: 0.2390 - val_loss: 2.0522 - val_accuracy: 0.2680
Epoch 4/5
16/16 [==============================] - 0s 19ms/step - loss: 2.0088 - accuracy: 0.2460 - val_loss: 1.9974 - val_accuracy: 0.2880
Epoch 5/5
16/16 [==============================] - 0s 10ms/step - loss: 1.9284 - accuracy: 0.2760 - val_loss: 1.9771 - val_accuracy: 0.2480


In [47]:
X_test_pca.shape

(250, 100)

In [48]:
y_test.shape

(250,)

In [49]:
# Evaluar el modelo con datos reducidos por PCA
test_loss_pca, test_accuracy_pca = model_pca.evaluate(X_test_pca, y_test)
print("Accuracy of CNN with PCA:", test_accuracy_pca)

8/8 [==============================] - 0s 3ms/step - loss: 1.9771 - accuracy: 0.2480
Accuracy of CNN with PCA: 0.24799999594688416
